In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

## get the classfiers

In [ ]:
### same preprocessing  as training
df = pd.read_csv('dataset_3_labels_merged.csv', sep=';')
texts = df['message_preprocessed']
dataf = pd.DataFrame({'text': texts})

max_words = 1000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(dataf['text'])
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(dataf['text'])
max_sequence_length = max(len(seq) for seq in sequences)

########  get the classifiers
model_rationale = load_model('bi_lstm_model_rationale.h5')
model_decision = load_model('bi_lstm_model_decision.h5')

# Generalization to other mm modules (slob.c)

## Test the classifiers

In [ ]:
df_slob = pd.read_csv('data_file_slob_preprocessed.csv' )
df_slob

,Unnamed: 0,commit ID,author name,committer name,message,URL,message_preprocessed
0,0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Vlastimil Babka,Vlastimil Babka,mm/slob: remove slob.c\n\nRemove the SLOB impl...,https://api.github.com/repos/torvalds/linux/gi...,mm/slob: remove slob.c
1,0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Vlastimil Babka,Vlastimil Babka,mm/slob: remove slob.c\n\nRemove the SLOB impl...,https://api.github.com/repos/torvalds/linux/gi...,Remove the SLOB implementation
2,0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Vlastimil Babka,Vlastimil Babka,mm/slob: remove slob.c\n\nRemove the SLOB impl...,https://api.github.com/repos/torvalds/linux/gi...,RIP SLOB allocator (2006 - 2023)
3,1,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,Kees Cook,Vlastimil Babka,slab: Introduce kmalloc_size_roundup()\n\nIn t...,https://api.github.com/repos/torvalds/linux/gi...,slab: Introduce kmalloc_size_roundup()
4,1,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,Kees Cook,Vlastimil Babka,slab: Introduce kmalloc_size_roundup()\n\nIn t...,https://api.github.com/repos/torvalds/linux/gi...,In the effort to help the compiler reason abou...
...,...,...,...,...,...,...,...
828,145,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Matt Mackall,Linus Torvalds,[PATCH] slob: introduce the SLOB allocator\n\n...,https://api.github.com/repos/torvalds/linux/gi...,It's been tested extensively in the Linux-tiny...
829,145,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Matt Mackall,Linus Torvalds,[PATCH] slob: introduce the SLOB allocator\n\n...,https://api.github.com/repos/torvalds/linux/gi...,I've also\nstress-tested it with make -j 8 co...
830,145,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Matt Mackall,Linus Torvalds,[PATCH] slob: introduce the SLOB allocator\n\n...,https://api.github.com/repos/torvalds/linux/gi...,Here's a comparison for otherwise identical bu...
831,145,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Matt Mackall,Linus Torvalds,[PATCH] slob: introduce the SLOB allocator\n\n...,https://api.github.com/repos/torvalds/linux/gi...,$ size vmlinux*\n text data bss d...


In [ ]:
X_test =  df_slob['message_preprocessed']

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(X_test.astype(str))
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)


# Making Decision predictions
y_pred_prob = model_decision.predict(padded_sequences) ## change here !!!!!
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions
y_pred = y_pred.flatten()


df_slob_predicted = pd.DataFrame({'commit ID': df_slob['commit ID']  , 'text': X_test, 'predicted_decision':y_pred})
df_slob_predicted


27/27 [==============================] - 1s 47ms/step


,commit ID,text,predicted_decision
0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,mm/slob: remove slob.c,1
1,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Remove the SLOB implementation,1
2,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,RIP SLOB allocator (2006 - 2023),0
3,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,slab: Introduce kmalloc_size_roundup(),1
4,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,In the effort to help the compiler reason abou...,1
...,...,...,...
828,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,It's been tested extensively in the Linux-tiny...,0
829,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,I've also\nstress-tested it with make -j 8 co...,1
830,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Here's a comparison for otherwise identical bu...,1
831,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,$ size vmlinux*\n text data bss d...,1


In [ ]:
# Making Rationale predictions
y_pred_prob = model_rationale.predict(padded_sequences) ## change here !!!!!
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions
y_pred = y_pred.flatten()


df_slob_predicted['predicted_rationale'] = y_pred
df_slob_predicted


27/27 [==============================] - 1s 30ms/step


,commit ID,text,predicted_decision,predicted_rationale
0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,mm/slob: remove slob.c,1,1
1,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Remove the SLOB implementation,1,1
2,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,RIP SLOB allocator (2006 - 2023),0,0
3,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,slab: Introduce kmalloc_size_roundup(),1,0
4,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,In the effort to help the compiler reason abou...,1,1
...,...,...,...,...
828,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,It's been tested extensively in the Linux-tiny...,0,0
829,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,I've also\nstress-tested it with make -j 8 co...,1,0
830,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Here's a comparison for otherwise identical bu...,1,1
831,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,$ size vmlinux*\n text data bss d...,1,1


In [ ]:
df_slob_predicted.to_csv('generalization_results_on_the_slob.csv', sep=';', index=False)

## Choose commits for validation

In [ ]:
import pandas as pd
df_slob_predicted = pd.read_csv('generalization_results_on_the_slob.csv' , sep=';', encoding ='latin-1')
df_slob_predicted

,commit ID,text,predicted_decision,predicted_rationale
0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,mm/slob: remove slob.c,1,1
1,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Remove the SLOB implementation,1,1
2,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,RIP SLOB allocator (2006 - 2023),0,0
3,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,slab: Introduce kmalloc_size_roundup(),1,0
4,C_kwDOACN7MtoAKDA1YTk0MDY1NmUxZWIyMDI2ZDllZTMx...,In the effort to help the compiler reason abou...,1,1
...,...,...,...,...
828,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,It's been tested extensively in the Linux-tiny...,0,0
829,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,I've also\nstress-tested it with make -j 8 co...,1,0
830,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,Here's a comparison for otherwise identical bu...,1,1
831,MDY6Q29tbWl0MjMyNTI5ODoxMGNlZjYwMjk1MDI5MTViZG...,$ size vmlinux*\n text data bss d...,1,1


choose a sample of 20 commits for validation

In [ ]:
unique_commits_IDs = df_slob_predicted['commit ID'].unique()
len(unique_commits_IDs)

146

In [ ]:
import random

random_commits_slob = random.choices(list(unique_commits_IDs), k=20)  # random 20 commits

random_commits_slob

['C_kwDOACN7MtoAKGJkNTNjZTRkYTI1MmRkYjFhZTQyNTdjMTY0ZjgwYWVhM2Q4YWI5MGM',
 'MDY6Q29tbWl0MjMyNTI5ODo3MDA5NmE1NjFkMWUwOTEyMGJhZTFmMjkzZjM2MzJjZWRiZmQ1YzY4',
 'MDY6Q29tbWl0MjMyNTI5ODpmMWI2ZWI2ZTZiZTE0OWI0MGViYjAxM2Y1YmZlMmFjODZiNmYxYzFi',
 'MDY6Q29tbWl0MjMyNTI5ODoyNDA4YzU1MDM3YzNmN2Q1MWE4YTEwMDAyNWM0NzU5NWU3MWI4Mzhj',
 'MDY6Q29tbWl0MjMyNTI5ODozYjBlZmRmYTFlNzE5MzAzNTM2YzA0ZDlhYmNhNDNhYmViNDBmODBh',
 'MDY6Q29tbWl0MjMyNTI5ODowMzljYTRlNzRhMWNmNjBiZDc0ODczMjRhNTY0ZWNmNWM5ODFmMjU0',
 'MDY6Q29tbWl0MjMyNTI5ODo1ZTE4ZTJiOGIzZDQ1M2U2OGFjY2MzZTI5NTY0M2ZlNGI1YmJjMjk1',
 'MDY6Q29tbWl0MjMyNTI5ODo1MWNjNTA2ODVhNDI3NWM2YTAyNjUzNjcwYWY5ZjEwOGE2NGUwMWNm',
 'MDY6Q29tbWl0MjMyNTI5ODo0YmE5YjlkMGJhMGE0OWQ5MWZhNjQxN2M3NTEwZWUzNmY0OGNmOTU3',
 'MDY6Q29tbWl0MjMyNTI5ODo4Y2Y5ODY0YjEzODI4NTFkOTBjN2M1MDVmODQ0MWM4OTI4ZjE0Njll',
 'MDY6Q29tbWl0MjMyNTI5ODo4NWJhOTRiYTA1OTIyOTYwNTNmN2YyODQ2ODEyMTczNDI0YWZlMWNi',
 'MDY6Q29tbWl0MjMyNTI5ODo1YmIxYmIzNTNjZmUzNDNmYzNjODRmYWYwNmY3MmJhMzA5ZmRlNTQx',
 'C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZ

In [ ]:
df_slob_predicted_sample = df_slob_predicted.loc[df_slob_predicted['commit ID'].isin(random_commits_slob)]
df_slob_predicted_sample

,commit ID,text,predicted_decision,predicted_rationale
0,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,mm/slob: remove slob.c,1,1
1,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,Remove the SLOB implementation,1,1
2,C_kwDOACN7MtoAKDY2MzBlOTUwZDUzMmZmYWJjZTNhNGJk...,RIP SLOB allocator (2006 - 2023),0,0
67,C_kwDOACN7MtoAKGJkNTNjZTRkYTI1MmRkYjFhZTQyNTdj...,mm/slob: make kmem_cache_boot static,0,0
68,C_kwDOACN7MtoAKGJkNTNjZTRkYTI1MmRkYjFhZTQyNTdj...,kmem_cache_boot is never accessed outside slob.c,0,0
...,...,...,...,...
652,MDY6Q29tbWl0MjMyNTI5ODo0YmE5YjlkMGJhMGE0OWQ5MW...,The object\npointer is placed before the kmem...,0,0
653,MDY6Q29tbWl0MjMyNTI5ODo0YmE5YjlkMGJhMGE0OWQ5MW...,"Convert\n ctor(void *object, struct kme...",0,0
660,MDY6Q29tbWl0MjMyNTI5ODoyNDA4YzU1MDM3YzNmN2Q1MW...,"{slub, slob}: use unlikely() for kfree(ZERO_OR...",0,0
661,MDY6Q29tbWl0MjMyNTI5ODoyNDA4YzU1MDM3YzNmN2Q1MW...,Considering kfree(NULL) would normally occur o...,1,1


In [ ]:
set(df_slob_predicted_sample['commit ID'].unique()) == set(random_commits_slob)

True

In [ ]:
df_slob_predicted_sample.to_csv('sample_of_generalization_results_on_the_slob.csv', sep=';', index=False)

## Alignement of Raters (post-validation)

In [ ]:
import pandas as pd

In [ ]:
df_b = pd.read_csv('Rater_1_sample_of_generalization_results_on_the_slob.csv' )
df_b = df_b.rename(columns={"Agree decision ? ": "Agree decision R1", "Agree rationale ? ": "Agree rationale R1"})

df_mi = pd.read_csv('Rater_2_sample_of_generalization_results_on_the_slob.csv' )
df_mi = df_mi.rename(columns={"Agree decision ? ": "Agree decision R2", "Agree rationale ? ": "Agree rationale R2"})

df_mo = pd.read_csv('Rater_3_sample_of_generalization_results_on_the_slob.csv' )
df_mo = df_mo.rename(columns={"Agree decision ? ": "Agree decision R3", "Agree rationale ? ": "Agree rationale R3"})


In [ ]:
df_slob_validation = df_b
df_slob_validation['Agree decision R2'] = df_mi['Agree decision R2']
df_slob_validation['Agree rationale R2'] = df_mi['Agree rationale R2']
df_slob_validation['Agree decision R3'] = df_mo['Agree decision R3']
df_slob_validation['Agree rationale R3'] = df_mo['Agree rationale R3']
df_slob_validation

In [ ]:
len(df_slob_validation[(df_slob_validation['Agree decision R1'] == "yes") & (df_slob_validation['Agree decision R2']=="yes") & (df_slob_validation['Agree decision R3'] == "yes")  ])

56

In [ ]:
len( df_slob_validation[(df_slob_validation['Agree rationale R1'] == "yes") & (df_slob_validation['Agree rationale R2']=="yes") & (df_slob_validation['Agree rationale R3'] == "yes")  ] )

51

In [ ]:
# yes and no, agreement
len(df_slob_validation[(df_slob_validation['Agree decision R1'] == df_slob_validation['Agree decision R2'] ) & (df_slob_validation['Agree decision R3'] == df_slob_validation['Agree decision R2'] )  ])

72

In [ ]:
# yes and no, agreement
len( df_slob_validation[(df_slob_validation['Agree rationale R1'] == df_slob_validation['Agree rationale R2'] )  & (df_slob_validation['Agree rationale R3'] == df_slob_validation['Agree rationale R2'] )  ] )

60

# Generalization to other components (drivers/acpi sub-project )

## Test the classifiers

In [ ]:
df_button = pd.read_csv('data_file_button_preprocessed.csv' )
df_button

,Unnamed: 0,commit ID,author name,committer name,message,URL,message_preprocessed
0,0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Ken Xue,Rafael J. Wysocki,ACPI: button: trigger wakeup key events\n\nAnd...,https://api.github.com/repos/torvalds/linux/gi...,ACPI: button: trigger wakeup key events
1,0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Ken Xue,Rafael J. Wysocki,ACPI: button: trigger wakeup key events\n\nAnd...,https://api.github.com/repos/torvalds/linux/gi...,Andorid can wakeup from various wakeup sources...
2,0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Ken Xue,Rafael J. Wysocki,ACPI: button: trigger wakeup key events\n\nAnd...,https://api.github.com/repos/torvalds/linux/gi...,"Regarding pressing acpi power button, it can r..."
3,0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Ken Xue,Rafael J. Wysocki,ACPI: button: trigger wakeup key events\n\nAnd...,https://api.github.com/repos/torvalds/linux/gi...,"So, send wakeup key event to user space during..."
4,1,C_kwDOACN7MtoAKGU0ZTYyZDVmZDhlY2IwZWE0MTc0YWZk...,Rafael J. Wysocki,Rafael J. Wysocki,ACPI: button: Use different notify handlers fo...,https://api.github.com/repos/torvalds/linux/gi...,ACPI: button: Use different notify handlers fo...
...,...,...,...,...,...,...,...
571,109,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linus Torvalds,Linus Torvalds,Linux-2.6.12-rc2\n\nInitial git repository bui...,https://api.github.com/repos/torvalds/linux/gi...,Linux-2.6.12-rc2
572,109,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linus Torvalds,Linus Torvalds,Linux-2.6.12-rc2\n\nInitial git repository bui...,https://api.github.com/repos/torvalds/linux/gi...,Initial git repository build
573,109,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linus Torvalds,Linus Torvalds,Linux-2.6.12-rc2\n\nInitial git repository bui...,https://api.github.com/repos/torvalds/linux/gi...,"I'm not bothering with the full history,\neven..."
574,109,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linus Torvalds,Linus Torvalds,Linux-2.6.12-rc2\n\nInitial git repository bui...,https://api.github.com/repos/torvalds/linux/gi...,"We can create a separate ""historical"" git\narc..."


In [ ]:
X_test =  df_button['message_preprocessed']

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(X_test.astype(str))
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)


# Making Decision predictions
y_pred_prob = model_decision.predict(padded_sequences) ## change here !!!!!
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions
y_pred = y_pred.flatten()


df_button_predicted = pd.DataFrame({'commit ID': df_button['commit ID'] , 'text': X_test, 'predicted_decision':y_pred})
df_button_predicted


18/18 [==============================] - 1s 31ms/step


,commit ID,text,predicted_decision
0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,ACPI: button: trigger wakeup key events,1
1,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Andorid can wakeup from various wakeup sources...,0
2,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,"Regarding pressing acpi power button, it can r...",0
3,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,"So, send wakeup key event to user space during...",1
4,C_kwDOACN7MtoAKGU0ZTYyZDVmZDhlY2IwZWE0MTc0YWZk...,ACPI: button: Use different notify handlers fo...,1
...,...,...,...
571,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linux-2.6.12-rc2,1
572,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Initial git repository build,1
573,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,"I'm not bothering with the full history,\neven...",1
574,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,"We can create a separate ""historical"" git\narc...",1


In [ ]:
# Making Rationale predictions
y_pred_prob = model_rationale.predict(padded_sequences) ## change here !!!!!
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions
y_pred = y_pred.flatten()


df_button_predicted['predicted_rationale'] = y_pred
df_button_predicted


18/18 [==============================] - 1s 32ms/step


,commit ID,text,predicted_decision,predicted_rationale
0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,ACPI: button: trigger wakeup key events,1,1
1,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Andorid can wakeup from various wakeup sources...,0,1
2,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,"Regarding pressing acpi power button, it can r...",0,0
3,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,"So, send wakeup key event to user space during...",1,0
4,C_kwDOACN7MtoAKGU0ZTYyZDVmZDhlY2IwZWE0MTc0YWZk...,ACPI: button: Use different notify handlers fo...,1,1
...,...,...,...,...
571,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linux-2.6.12-rc2,1,1
572,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Initial git repository build,1,1
573,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,"I'm not bothering with the full history,\neven...",1,0
574,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,"We can create a separate ""historical"" git\narc...",1,1


In [ ]:
df_button_predicted.to_csv('generalization_results_on_the_button.csv', sep=';', index=False)

## Choose 20 commits for validation

In [ ]:
import pandas as pd
df_button_predicted = pd.read_csv('generalization_results_on_the_button.csv' , sep=';', encoding ='latin-1')
df_button_predicted

,commit ID,text,predicted_decision,predicted_rationale
0,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,ACPI: button: trigger wakeup key events,1,1
1,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,Andorid can wakeup from various wakeup sources...,0,1
2,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,"Regarding pressing acpi power button, it can r...",0,0
3,C_kwDOACN7MtoAKDE2ZjcwZmVhYWJlOWZkZTBhZjcwM2Yy...,"So, send wakeup key event to user space during...",1,0
4,C_kwDOACN7MtoAKGU0ZTYyZDVmZDhlY2IwZWE0MTc0YWZk...,ACPI: button: Use different notify handlers fo...,1,1
...,...,...,...,...
571,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Linux-2.6.12-rc2,1,1
572,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,Initial git repository build,1,1
573,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,"I'm not bothering with the full history,\neven...",1,0
574,MDY6Q29tbWl0MjMyNTI5ODoxZGExNzdlNGMzZjQxNTI0ZT...,"We can create a separate ""historical"" git\narc...",1,1


In [ ]:
unique_commits_IDs_button = df_button_predicted['commit ID'].unique()
len(unique_commits_IDs_button)

110

In [ ]:
import random

random_commits_button = random.choices(list(unique_commits_IDs_button), k=20)  # random 20 commits

random_commits_button

['MDY6Q29tbWl0MjMyNTI5ODoyMTk4OGE4ZTUxNDc5Y2VmZmU3YjA1NjhiMTcwZWZmYWJiNzA4ZGZl',
 'MDY6Q29tbWl0MjMyNTI5ODphYzFjYzZiNDg1NmZmYTdlY2I4MThiM2NlYjgyYmZjNDM1OTdkNjEz',
 'MDY6Q29tbWl0MjMyNTI5ODoxM2MxOTljMGQwY2Y3OGIyNzU5Mjk5MTEyOWZiOGNiY2ZjNTE2NGRl',
 'MDY6Q29tbWl0MjMyNTI5ODo0YmU0NGZjZDNiZjY0OGI3ODJmNDQ2MGZkMDZkZmFlNmM0MmRlZDRi',
 'MDY6Q29tbWl0MjMyNTI5ODoyYjYwMjJhMzJlMWM2ZTc0MGE1OTI5OTUxZDlkOGNiZWQzYjE2MmIx',
 'MDY6Q29tbWl0MjMyNTI5ODozNTQwYzMyYTlhZTRjYjIzYWI3MGY3Nzk4ZjQ1YWZmYzAyNzYyZmE3',
 'MDY6Q29tbWl0MjMyNTI5ODo5MTJiNzQyN2ZjMTM0NjdiYWMwNDJjYzcyZjJlY2NiZGJjMzlkNzcz',
 'MDY6Q29tbWl0MjMyNTI5ODo1MWZhYzgzODhhMDMyNWE0M2YwYWU2NzQ1M2VjZTJjMzczZTJlYzI4',
 'C_kwDOACN7MtoAKDZjMGViNWJhMzUwMGY2ZGEzNjczNTFmZjNjNDQ1MmMwMjljYjcyZmE',
 'MDY6Q29tbWl0MjMyNTI5ODo3ZGFhYTA2MzU3YmY3ZjE4NzRiNjJiYjFlYTlkNjZhNTFkNGU1Njdl',
 'MDY6Q29tbWl0MjMyNTI5ODplMzQ2ZDBjZjJjMGEyZGM5ZTYzZDViOTA4MjRiYmU1YWMwY2M0M2Uy',
 'MDY6Q29tbWl0MjMyNTI5ODpkZjMxNmU5MzkxMDBlNzg5YjNjNWQ0ZDEwMjYxOWNjZjU4MzRiZDAw',
 'MDY6Q29tbWl0MjMyNTI5ODoxMzZhNGRmZ

In [ ]:
df_button_predicted_sample = df_button_predicted.loc[df_button_predicted['commit ID'].isin(random_commits_button)]
df_button_predicted_sample

,commit ID,text,predicted_decision,predicted_rationale
12,C_kwDOACN7MtoAKDZjMGViNWJhMzUwMGY2ZGEzNjczNTFm...,ACPI: make remove callback of ACPI driver void,1,0
13,C_kwDOACN7MtoAKDZjMGViNWJhMzUwMGY2ZGEzNjczNTFm...,"For bus-based driver, device removal is implem...",0,0
14,C_kwDOACN7MtoAKDZjMGViNWJhMzUwMGY2ZGEzNjczNTFm...,1 device_remove()->\n2 bus->remove()->\n3 ...,1,1
15,C_kwDOACN7MtoAKDZjMGViNWJhMzUwMGY2ZGEzNjczNTFm...,"In that case, commit fc7a6209d571\n(""bus: Make...",1,0
16,C_kwDOACN7MtoAKDZjMGViNWJhMzUwMGY2ZGEzNjczNTFm...,Now we have the situation that both 1 & 2 of c...,0,0
...,...,...,...,...
528,MDY6Q29tbWl0MjMyNTI5ODo3Y2RhOTNlMDA4ZTFhNDc3OT...,ACPI: delete extra #defines in /drivers/acpi/ ...,0,1
529,MDY6Q29tbWl0MjMyNTI5ODo3Y2RhOTNlMDA4ZTFhNDc3OT...,Cosmetic only,1,1
530,MDY6Q29tbWl0MjMyNTI5ODo3Y2RhOTNlMDA4ZTFhNDc3OT...,"Except in a single case, #define ACPI_*_DRIVER...",0,1
565,MDY6Q29tbWl0MjMyNTI5ODo0YmU0NGZjZDNiZjY0OGI3OD...,[ACPI] Lindent all ACPI files,0,0


In [ ]:
set(df_button_predicted_sample['commit ID'].unique()) == set(random_commits_button)

True

In [ ]:
df_button_predicted_sample.to_csv('sample_of_generalization_results_on_the_button.csv', sep=';', index=False)

## Alignment of rater (post-validation)

In [ ]:
import pandas as pd

In [ ]:
df_b = pd.read_csv('Rater_1_sample_of_generalization_results_on_the_button.csv' )
df_b = df_b.rename(columns={"Agree decision ? ": "Agree decision R1", "Agree rationale ?": "Agree rationale R1"})

df_mi = pd.read_csv('Rater_2_sample_of_generalization_results_on_the_button.csv' )
df_mi = df_mi.rename(columns={"Agree decision ? ": "Agree decision R2", "Agree rationale ?": "Agree rationale R2"})

df_mo = pd.read_csv('Rater_3_sample_of_generalization_results_on_the_button.csv' )
df_mo = df_mo.rename(columns={"Agree decision ? ": "Agree decision R3", "Agree rationale ?": "Agree rationale R3"})

In [ ]:
df_button_validation = df_b
df_button_validation['Agree decision R2'] = df_mi['Agree decision R2']
df_button_validation['Agree rationale R2'] = df_mi['Agree rationale R2']
df_button_validation['Agree decision R3'] = df_mo['Agree decision R3']
df_button_validation['Agree rationale R3'] = df_mo['Agree rationale R3']
df_button_validation


In [ ]:
df_button_validation[(df_button_validation['Agree decision R1'] == "yes") & (df_button_validation['Agree decision R2']=="yes") & (df_button_validation['Agree decision R3'] == "yes")  ]

In [ ]:
len(df_button_validation[(df_button_validation['Agree decision R1'] == "yes") & (df_button_validation['Agree decision R2']=="yes") & (df_button_validation['Agree decision R3'] == "yes")  ])

52

In [ ]:
len(df_button_validation[(df_button_validation['Agree rationale R1'] == "yes") & (df_button_validation['Agree rationale R2']=="yes") & (df_button_validation['Agree rationale R3'] == "yes")  ])

51

In [ ]:
# yes and no, agreement
len(df_button_validation[(df_button_validation['Agree decision R1'] == df_button_validation['Agree decision R2'] ) & (df_button_validation['Agree decision R3'] == df_button_validation['Agree decision R2'] )  ])

69

In [ ]:
# yes and no, agreement
len(df_button_validation[(df_button_validation['Agree rationale R1'] == df_button_validation['Agree rationale R2'] ) & (df_button_validation['Agree rationale R3'] == df_button_validation['Agree rationale R2'] )  ])

55

# Generalization to other projects (What makes a good commit message dataset)

## Prepapre the data

 * It contains meta-information of 1649 labeled commit messages.
    * label = 0 means a commit message contains "Why and What".
    * label = 1 means a commit message  contains "Neither Why nor What".
    * label = 2 means a commit message  contains "No What" :  “No What”(only Why, but no What);
    * label = 3 means a commit message  contains "No Why". : “No Why” (only What information, but no Why);
  
    * if_mulit_commit = 1 means a commit is non-atomic.
    * new_message1 means a message after preprocessing.


In [ ]:
df_commit = pd.read_csv('sampled messages_what_makes_a_good_commit.csv', encoding='latin-1')
df_commit

,id,repo_id,label,url,if_mulit_commit,message,new_message1,authorEmail,commitDate,Unnamed: 9
0,3079,2,0,https://github.com/apache/dubbo/commit/1b2e6dc...,1,remove some magic value (#4752) <enter> <ente...,remove some magic value ( <pr_link> ) <enter>...,developer128,2019-08-06T07:23:45Z,NaN
1,5904,4,0,https://github.com/square/okhttp/commit/2fd0da...,,Update concurrency.md (#6290),Update concurrency.md ( <pr_link> ),developer129,2020-09-30T16:01:02Z,NaN
2,4224,2,0,https://github.com/apache/dubbo/commit/9e9e778...,,Optimize_hessian_desr_performance (#1705),Optimize_hessian_desr_performance ( <pr_link> ),developer130,2018-04-26T10:51:02Z,NaN
3,5382,3,0,https://github.com/square/retrofit/commit/0754...,,Add @PartMap annotation.,Add <iden> annotation.,developer131,2014-05-01T17:16:02Z,NaN
4,6983,4,3,https://github.com/square/okhttp/commit/c310d2...,,workaround broken system dns behaviour,workaround broken system <file_name> behaviour,developer132,2017-11-17T11:02:26Z,NaN
...,...,...,...,...,...,...,...,...,...,...
1644,3452,2,0,https://github.com/apache/dubbo/commit/789775b...,,Change CopyOnWriteArrayList to regular list in...,Change <iden> to regular list in <file_name> ...,developer335,2019-05-15T03:05:48Z,NaN
1645,3786,2,2,https://github.com/apache/dubbo/commit/84d355f...,,code rule (#3651),code rule ( <pr_link> ),developer336,2019-04-12T03:37:03Z,NaN
1646,3497,2,0,https://github.com/apache/dubbo/commit/3bdc316...,,replace Random with ThreadLocalRandom (#2433),replace <iden> with <iden> ( <pr_link> ),developer337,2018-09-05T03:31:52Z,NaN
1647,13889,5,0,https://github.com/spring-projects/spring-boot...,,Update production-ready-features.adoc <enter> ...,Update <file_name> <enter> Replace line break...,developer338,2016-12-08T11:15:43Z,NaN


In [ ]:
df_commit['if_mulit_commit'].value_counts()

     1597
1      52
Name: if_mulit_commit, dtype: int64

In [ ]:
df_commit = df_commit.drop(df_commit[df_commit.if_mulit_commit == '1' ].index)
df_commit

,id,repo_id,label,url,if_mulit_commit,message,new_message1,authorEmail,commitDate,Unnamed: 9
1,5904,4,0,https://github.com/square/okhttp/commit/2fd0da...,,Update concurrency.md (#6290),Update concurrency.md ( <pr_link> ),developer129,2020-09-30T16:01:02Z,NaN
2,4224,2,0,https://github.com/apache/dubbo/commit/9e9e778...,,Optimize_hessian_desr_performance (#1705),Optimize_hessian_desr_performance ( <pr_link> ),developer130,2018-04-26T10:51:02Z,NaN
3,5382,3,0,https://github.com/square/retrofit/commit/0754...,,Add @PartMap annotation.,Add <iden> annotation.,developer131,2014-05-01T17:16:02Z,NaN
4,6983,4,3,https://github.com/square/okhttp/commit/c310d2...,,workaround broken system dns behaviour,workaround broken system <file_name> behaviour,developer132,2017-11-17T11:02:26Z,NaN
5,3817,2,0,https://github.com/apache/dubbo/commit/7f262f9...,,simplify map empty judgment (#3376),simplify map empty judgment ( <pr_link> ),developer133,2019-01-29T05:38:51Z,NaN
...,...,...,...,...,...,...,...,...,...,...
1644,3452,2,0,https://github.com/apache/dubbo/commit/789775b...,,Change CopyOnWriteArrayList to regular list in...,Change <iden> to regular list in <file_name> ...,developer335,2019-05-15T03:05:48Z,NaN
1645,3786,2,2,https://github.com/apache/dubbo/commit/84d355f...,,code rule (#3651),code rule ( <pr_link> ),developer336,2019-04-12T03:37:03Z,NaN
1646,3497,2,0,https://github.com/apache/dubbo/commit/3bdc316...,,replace Random with ThreadLocalRandom (#2433),replace <iden> with <iden> ( <pr_link> ),developer337,2018-09-05T03:31:52Z,NaN
1647,13889,5,0,https://github.com/spring-projects/spring-boot...,,Update production-ready-features.adoc <enter> ...,Update <file_name> <enter> Replace line break...,developer338,2016-12-08T11:15:43Z,NaN


In [ ]:
df_commit = df_commit.drop(columns=['authorEmail', 'commitDate', 'message', 'id','Unnamed: 9', 'if_mulit_commit'])
df_commit

,repo_id,label,url,new_message1
1,4,0,https://github.com/square/okhttp/commit/2fd0da...,Update concurrency.md ( <pr_link> )
2,2,0,https://github.com/apache/dubbo/commit/9e9e778...,Optimize_hessian_desr_performance ( <pr_link> )
3,3,0,https://github.com/square/retrofit/commit/0754...,Add <iden> annotation.
4,4,3,https://github.com/square/okhttp/commit/c310d2...,workaround broken system <file_name> behaviour
5,2,0,https://github.com/apache/dubbo/commit/7f262f9...,simplify map empty judgment ( <pr_link> )
...,...,...,...,...
1644,2,0,https://github.com/apache/dubbo/commit/789775b...,Change <iden> to regular list in <file_name> ...
1645,2,2,https://github.com/apache/dubbo/commit/84d355f...,code rule ( <pr_link> )
1646,2,0,https://github.com/apache/dubbo/commit/3bdc316...,replace <iden> with <iden> ( <pr_link> )
1647,5,0,https://github.com/spring-projects/spring-boot...,Update <file_name> <enter> Replace line break...


In [ ]:
df_commit['Decision']=df_commit['label'].apply(lambda x: 1 if (x==0 or x == 3) else 0)  # both + no why
df_commit['Rationale']=df_commit['label'].apply(lambda x: 1 if (x==0 or x == 2) else 0)  # both + no what
df_commit

,repo_id,label,url,new_message1,Decision,Rationale
1,4,0,https://github.com/square/okhttp/commit/2fd0da...,Update concurrency.md ( <pr_link> ),1,1
2,2,0,https://github.com/apache/dubbo/commit/9e9e778...,Optimize_hessian_desr_performance ( <pr_link> ),1,1
3,3,0,https://github.com/square/retrofit/commit/0754...,Add <iden> annotation.,1,1
4,4,3,https://github.com/square/okhttp/commit/c310d2...,workaround broken system <file_name> behaviour,1,0
5,2,0,https://github.com/apache/dubbo/commit/7f262f9...,simplify map empty judgment ( <pr_link> ),1,1
...,...,...,...,...,...,...
1644,2,0,https://github.com/apache/dubbo/commit/789775b...,Change <iden> to regular list in <file_name> ...,1,1
1645,2,2,https://github.com/apache/dubbo/commit/84d355f...,code rule ( <pr_link> ),0,1
1646,2,0,https://github.com/apache/dubbo/commit/3bdc316...,replace <iden> with <iden> ( <pr_link> ),1,1
1647,5,0,https://github.com/spring-projects/spring-boot...,Update <file_name> <enter> Replace line break...,1,1


Note: These are at the commit level!



## Test the classifiers

In [ ]:
X_test =  df_commit['new_message1']

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(X_test.astype(str))
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)


# Making Decision predictions
y_pred_prob = model_decision.predict(padded_sequences) ## change here !!!!!
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions
y_pred = y_pred.flatten()

df_commit['predicted_decision'] = y_pred


# Making Rationale predictions
y_pred_prob = model_rationale.predict(padded_sequences) ## change here !!!!!
y_pred = (y_pred_prob > 0.5).astype(int)  # Threshold probabilities to get binary predictions
y_pred = y_pred.flatten()

df_commit['predicted_rationale'] = y_pred


df_commit


50/50 [==============================] - 2s 31ms/step


,repo_id,label,url,new_message1,Decision,Rationale,predicted_decision,predicted_rationale
1,4,0,https://github.com/square/okhttp/commit/2fd0da...,Update concurrency.md ( <pr_link> ),1,1,1,1
2,2,0,https://github.com/apache/dubbo/commit/9e9e778...,Optimize_hessian_desr_performance ( <pr_link> ),1,1,1,0
3,3,0,https://github.com/square/retrofit/commit/0754...,Add <iden> annotation.,1,1,1,1
4,4,3,https://github.com/square/okhttp/commit/c310d2...,workaround broken system <file_name> behaviour,1,0,1,1
5,2,0,https://github.com/apache/dubbo/commit/7f262f9...,simplify map empty judgment ( <pr_link> ),1,1,1,1
...,...,...,...,...,...,...,...,...
1644,2,0,https://github.com/apache/dubbo/commit/789775b...,Change <iden> to regular list in <file_name> ...,1,1,1,1
1645,2,2,https://github.com/apache/dubbo/commit/84d355f...,code rule ( <pr_link> ),0,1,1,0
1646,2,0,https://github.com/apache/dubbo/commit/3bdc316...,replace <iden> with <iden> ( <pr_link> ),1,1,1,0
1647,5,0,https://github.com/spring-projects/spring-boot...,Update <file_name> <enter> Replace line break...,1,1,1,1


In [ ]:
df_commit.to_csv('generalization_results_on_the_commit_messages_paper.csv', sep=';', index=False)

In [ ]:
import pandas as pd
df_commit = pd.read_csv('generalization_results_on_the_commit_messages_paper.csv' , sep=';', encoding ='latin-1')
df_commit

,repo_id,label,url,new_message1,Decision,Rationale,predicted_decision,predicted_rationale
0,4,0,https://github.com/square/okhttp/commit/2fd0da...,Update concurrency.md ( <pr_link> ),1,1,1,1
1,2,0,https://github.com/apache/dubbo/commit/9e9e778...,Optimize_hessian_desr_performance ( <pr_link> ),1,1,1,0
2,3,0,https://github.com/square/retrofit/commit/0754...,Add <iden> annotation.,1,1,1,1
3,4,3,https://github.com/square/okhttp/commit/c310d2...,workaround broken system <file_name> behaviour,1,0,1,1
4,2,0,https://github.com/apache/dubbo/commit/7f262f9...,simplify map empty judgment ( <pr_link> ),1,1,1,1
...,...,...,...,...,...,...,...,...
1592,2,0,https://github.com/apache/dubbo/commit/789775b...,Change <iden> to regular list in <file_name> ...,1,1,1,1
1593,2,2,https://github.com/apache/dubbo/commit/84d355f...,code rule ( <pr_link> ),0,1,1,0
1594,2,0,https://github.com/apache/dubbo/commit/3bdc316...,replace <iden> with <iden> ( <pr_link> ),1,1,1,0
1595,5,0,https://github.com/spring-projects/spring-boot...,Update <file_name> <enter> Replace line break...,1,1,1,1


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = accuracy_score(df_commit['Decision'], df_commit['predicted_decision'])
precision = precision_score(df_commit['Decision'], df_commit['predicted_decision'])
recall = recall_score(df_commit['Decision'], df_commit['predicted_decision'])
f1 = f1_score(df_commit['Decision'], df_commit['predicted_decision'])

print("Test Metrics Decision:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

#############
accuracy = accuracy_score(df_commit['Rationale'], df_commit['predicted_rationale'])
precision = precision_score(df_commit['Rationale'], df_commit['predicted_rationale'])
recall = recall_score(df_commit['Rationale'], df_commit['predicted_rationale'])
f1 = f1_score(df_commit['Rationale'], df_commit['predicted_rationale'])

print("\n Test Metrics Rationale:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Test Metrics Decision:
Accuracy: 0.5616781465247339
Precision: 0.8280773143438453
Recall: 0.6052044609665428
F1 Score: 0.6993127147766323

 Test Metrics Rationale:
Accuracy: 0.6393237319974953
Precision: 0.7272727272727273
Recall: 0.7667269439421338
F1 Score: 0.7464788732394367
